**WOLF AND KANTZ ALGORITHM FOR TEMPORAL NETWORK**

In the next cell we ask you to insert the name of the input and output file.
The input file should be a time series of numpy Matrices (extension: .npy).

In [ ]:
file_name_input=input("Please enter input file name: ")
file_name_output=input("Please enter output file name: ")

The next cell is the code, the output file is a txt containing two columns with title W and R.
Here, W is the local expansion rate and R is the corresponding R^2. 
The Maximum Lyapunov exponent of the series is the mean of the W.

In [ ]:
import time
import math
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np
import networkx as nx
import scipy
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import ks_2samp


data = np.load(file_name_input)
nodes=len(data[0])

T=len(data) #selected time 
lT=0
def distance(x,y):
    N=len(x) 
    d=np.sum(abs(x-y)/(N*(N-1))) #difference between adjacency matrix
    return d

def Rsquared(x,y,y_tot,f): # compute the R-squared
  
    z=np.linspace(0,len(y_tot)-1,len(y_tot))
    y_bar = np.mean(y)
    ss_tot = ((y-y_bar)**2).sum()
    ss_res = ((y-f)**2).sum()
    return 1 - (ss_res/ss_tot)

def longst_nz(input_list: list) -> list:


    max_subset = []
    current_max_subset = []

    for number in input_list:
        if number > 0:
            current_max_subset.append(number)
        else:
            if len(current_max_subset) > len(max_subset):
                max_subset = current_max_subset
            current_max_subset = []

    return np.array(max_subset)
  

def best_fit(y_nl,window): # perform the fit over a window of points 
    y=np.log(longst_nz([0,0,*y_nl,0])) 
    best_R=0
    i=0
    x=np.linspace(0,len(y)-1,len(y)) 
    a,b=np.polyfit(x[:i+window],y[:i+window],1)
    f=a*x+b
    a_old=a
    R=Rsquared(x[:i+window], y[:i+window],y,f[:i+window])
    best_R=R

    while  i<len(y)-window and R>=best_R:
        a_old=a
        best_R=R
        i=i+1
        a,b=np.polyfit(x[:i+window],y[:i+window],1)
        f=a*x+b
        R=Rsquared(x[:i+window], y[:i+window],y,f[:i+window])
    
    return a_old,best_R,i-1

#WOLF algorithm

def Wolf_n(Trajectory, Time, transient,tau,w,eps):

    i=transient   # Index for referring to the initial point  
    dista=[] # Array for distances 
    
    S=[] # Initialize the list of local expansion rates
    R=[]
    while i< Time-tau-1 : # check to do not exceed the time limit
            
        xi=Trajectory[i] # we take the value of the evolution of the initial point at time transient+ tau*iteration
        d_old_j=closest(i,Trajectory,Time-tau-1,eps,250)            #compute the minimum distance between xi and another point in the trajectory
    
        xk=Trajectory[d_old_j] # the point associated tot the minimum distance
        dista=[] 
        z=0
        #dista.append(distance(xk,xi))  #exclude the first point in distance (need to do it because we don't know about randomness)
        while  z<tau-1: # Advance until tau
            z=z+1 # advancing in time
            ds=distance(xk,xi) # save previous distance
            xi=Trajectory[i+z] # compute the evolution of xi
            xk=Trajectory[d_old_j+z] # compute the evolution of xk
            dista.append(distance(xk,xi)) #append the distance at time z
            
        s,r,inp=best_fit(dista,w) # compute the fit on the array of distances
        S.append(s)
        R.append(r)  

        i=i+tau #the initial point advancing in time 
        
    return S,R, np.mean(S)

#FINDING CLOSER POINT
from scipy.signal import argrelextrema

def closest(k, Traj,T, eps, corr_len): # need the initial index, the trajectory, transient time, Total time
    d=[]
    dj=[]
    di=[]
    dij=[]
    for j in range(0,T):
        dist=distance(Traj[k], Traj[j])
        #plt.scatter(j,dist, color='black')
        d.append(dist)
        dj.append(j) 
 
    indeces=np.array(argrelextrema(np.array(d), np.less)[0])
    
    for i in indeces:
        if d[i]>eps and abs(i-k)>corr_len:
            di.append(d[i])
            dij.append(dj[i])
    
    index=dij[np.where(di==np.amin(di))[0][0]]

    return index # return 
start=time.time()

#run the Wolf algorithm
W,R,w=Wolf_n(data,len(data),0,100,5,0.)


import sys
np.set_printoptions(threshold=sys.maxsize)
original_stdout = sys.stdout # Save a reference to the original standard output

with open(file_name_output, 'w+') as f:
    sys.stdout = f # Change the standard output to the file we created.
   
    print('W R')
    
    for i in range(len(W)):
        print(W[i],R[i], sep = " ")
    print('')
    print('time of the program=',(time.time()-start)/60)
    sys.stdout = original_stdout # Reset the standard output to its original value
